# Classifying Risks Leveraging Poisson Regression Model

- The frequency of claim is the target variable here.
- Minimal risk rating factor that can be leveraged to classify risks into frequency bands will be a good outcome to nail down
- Make the model an inference service that can bedeployed

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## But First, Shall We Prepare the Dataset 

- Claims data includes yearly totals and breakdowns by claim type.
- Aggregate the claims data to obtain the claims frequency unique to each ID and year
- Merge result with rating factors on (ID, year).
- Assign a frequency of 0 to policies with no recorded claims.

In [3]:
rating_factors = pd.read_csv('../data/input/exp/Motor vehicle insurance data.csv', delimiter=";")
claims =  pd.read_csv('../data/input/exp/sample type claim.csv', delimiter=';')

In [4]:
claims_frequency  = (
    claims
    .groupby(['ID', 'Cost_claims_year'])
    .agg({
        'Cost_claims_by_type': 'count'})
    .rename(columns={'Cost_claims_by_type': 'claims_frequency'})
    .reset_index()
)

In [ ]:
dataset = (
    pd.merge(
        left=rating_factors,
        right=claims_frequency,
        how='left',
        on=['ID', 'Cost_claims_year']
    ).fillna
)
dataset.describe()

,ID,Distribution_channel,Seniority,Policies_in_force,Max_policies,Max_products,Lapse,Payment,Premium,Cost_claims_year,...,Area,Second_driver,Year_matriculation,Power,Cylinder_capacity,Value_vehicle,N_doors,Length,Weight,claims_frequency
count,105555.000000,105555.000000,105555.000000,105555.000000,105555.000000,105555.000000,105555.000000,105555.000000,105555.000000,105555.000000,...,105555.000000,105555.000000,105555.000000,105555.000000,105555.000000,105555.000000,105555.000000,95226.000000,105555.000000,5258.000000
mean,26271.286789,0.451310,6.696604,1.455649,1.837232,1.065842,0.221837,0.319180,315.892557,153.557305,...,0.273895,0.123708,2004.728038,92.682611,1617.759367,18413.657243,4.067898,4.252007,1191.262422,1.401483
std,15388.309324,0.497626,6.263911,0.928427,1.155536,0.267807,0.464858,0.466161,140.927969,1477.112362,...,0.445958,0.329250,6.767037,37.012645,604.697382,9135.074235,1.511839,0.393220,458.081834,0.750559
min,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,40.140000,0.000000,...,0.000000,0.000000,1950.000000,0.000000,49.000000,270.460000,0.000000,1.978000,43.000000,1.000000
25%,12925.000000,0.000000,3.000000,1.000000,1.000000,1.000000,0.000000,0.000000,241.610000,0.000000,...,0.000000,0.000000,2001.000000,75.000000,1390.000000,13127.210000,3.000000,3.999000,1043.000000,1.000000
50%,26082.000000,0.000000,4.000000,1.000000,2.000000,1.000000,0.000000,0.000000,292.280000,0.000000,...,0.000000,0.000000,2005.000000,90.000000,1598.000000,17608.770000,5.000000,4.230000,1205.000000,1.000000
75%,39754.000000,1.000000,9.000000,2.000000,2.000000,1.000000,0.000000,1.000000,361.640000,0.000000,...,1.000000,0.000000,2008.000000,110.000000,1910.000000,22595.000000,5.000000,4.443000,1388.000000,2.000000
max,53502.000000,1.000000,40.000000,17.000000,17.000000,4.000000,7.000000,1.000000,2993.340000,260853.240000,...,1.000000,1.000000,2018.000000,580.000000,7480.000000,220675.800000,6.000000,8.218000,7300.000000,9.000000


## Claims rate per unit exposure
- The data is structured such that each record is policy that runs for a year, as such let's calculate the claims rate per unit exposure (policy length)
- Creating matrix a using - Age, driving lenght, insurable object type (e.g van, motorcycle etc.)
    - Age banded into 3 levels of 0-35, 36-70, >70 as 0,1,2
    - Driving length banded into 2 levels of less than 5 years of experience and greater than 5 years experience as 0,1
    - Type risk has levels 1,2,3,4 representing motorbikes, van, passengers and agricultural vehicles respectively

In [41]:
rating_factors_a = (
    dataset
    .copy()[['Date_driving_licence','Date_birth','Type_risk', 'claims_frequency','Cost_claims_year']]
    .fillna({'Date_driving_licence': pd.NaT,  'claims_frequency': 0})
    .assign(age_band=lambda x: pd.cut((pd.Timestamp.now() - pd.to_datetime(x['Date_birth'], dayfirst=True)).dt.days // 365, 
                                 bins=[0, 35, 70, float('inf')], 
                                 labels=[0, 1, 2], 
                                 include_lowest=True).astype(int))
    .assign(driving_length=lambda x:((pd.Timestamp.now() - pd.to_datetime(x['Date_driving_licence'], dayfirst=True)).dt.days // 365 >= 5).astype(int))
    .drop(columns=['Date_driving_licence', 'Date_birth'], axis=1)
)
rating_factors_a

,Type_risk,claims_frequency,Cost_claims_year,age_band,driving_length
0,1,0.0,0.0,1,1
1,1,0.0,0.0,1,1
2,1,0.0,0.0,1,1
3,1,0.0,0.0,1,1
4,1,0.0,0.0,1,1
...,...,...,...,...,...
105550,3,0.0,0.0,1,1
105551,3,0.0,0.0,1,1
105552,3,0.0,0.0,1,1
105553,2,0.0,0.0,2,1


In [42]:
rating_factors_a_matrix = (
    rating_factors_a
    .groupby(['age_band', 'driving_length', 'Type_risk'])
    .agg({'claims_frequency':'sum', 'Cost_claims_year':'sum'})
    .assign(claims_rate_per_unit_exposure=lambda x:(x['claims_frequency']/x['Cost_claims_year']))
    .reset_index()
)
rating_factors_a_matrix

,age_band,driving_length,Type_risk,claims_frequency,Cost_claims_year,claims_rate_per_unit_exposure
0,0,1,1,27.0,51431.88,0.000525
1,0,1,2,24.0,49850.71,0.000481
2,0,1,3,496.0,1423505.43,0.000348
3,0,1,4,0.0,0.00,NaN
4,1,1,1,209.0,224939.07,0.000929
5,1,1,2,822.0,1917519.21,0.000429
6,1,1,3,5111.0,11103401.38,0.000460
7,1,1,4,0.0,192.67,0.000000
8,2,1,1,16.0,32685.29,0.000490
9,2,1,2,141.0,230383.40,0.000612
